## Validation of model predictions for carbon utilization

In [1]:
import cobra
import pandas as pd
import os

pd.set_option('display.max_rows', None)

# Call the metabolic models
s12 = cobra.io.read_sbml_model('iSH1474.xml')
kt2440 = cobra.io.read_sbml_model('iJN1463.xml')

# Open the PM experimental result file
pm = pd.read_excel('PM_result.xlsx')

In [2]:
# Function definition

# Function that compares experimental and simulation results
def check (x, y):
    if x == '+':
        if y == '+':
            ans = 'TP'
        else:
            ans = 'FN'
    else:
        if y == '+':
            ans = 'FP'
        else:
            ans = 'TN'
            
    return ans

# Function that simulate the growh of model in each sole carbon source
def growth_simulation (model):

    metabolites = pm['metabolites id'].values.tolist()
    exp_result = pm['s12 growth'].values.tolist()
    
    simulation_result = []
    min_growth = model.slim_optimize()*0.05

    for i in range(len(metabolites)):
        m = metabolites[i]
        x = exp_result[i]
        
        if m == 'x':
            y = '-'
        else:
            with model:
                m_id = 'EX_'+m+'_e'
            
                model.reactions.get_by_id('EX_glc__D_e').bounds = (0,0)
                
                try:
                    model.reactions.get_by_id(m_id).lower_bound = -6
            
                    simulation_growth = model.slim_optimize()
            
                    if simulation_growth > min_growth:
                        y = '+'
                    else:
                        y = '-'
                    
                except:
                    y = '-'
                    
                    
        simulation_result.append(check (x, y))
        
                    
    return simulation_result

In [3]:
for gem in ['s12','kt2440']:
    model = globals()[gem]
    simulation_result = growth_simulation(model)
    pm[gem+' simulation result'] = simulation_result

In [4]:
# To generate Figure 4A in the reference paper

data = {'TP':[],'TN':[],'FP':[],'FN':[]}

data['TP'].append(pm['s12 simulation result'].to_list().count('TP'))
data['TN'].append(pm['s12 simulation result'].to_list().count('TN'))
data['FP'].append(pm['s12 simulation result'].to_list().count('FP'))
data['FN'].append(pm['s12 simulation result'].to_list().count('FN'))

data['TP'].append(pm['kt2440 simulation result'].to_list().count('TP'))
data['TN'].append(pm['kt2440 simulation result'].to_list().count('TN'))
data['FP'].append(pm['kt2440 simulation result'].to_list().count('FP'))
data['FN'].append(pm['kt2440 simulation result'].to_list().count('FN'))

display(pd.DataFrame(data,index=['iSH1474 (S12)','iJN1462 (KT2440)']))
print('iSH1474 accuracy:',(pm[['s12 simulation result']].value_counts().TP + pm[['s12 simulation result']].value_counts().TN)/203 * 100)
print('iJN1462 accuracy:',(pm[['kt2440 simulation result']].value_counts().TP + pm[['kt2440 simulation result']].value_counts().TN)/203 * 100)

,TP,TN,FP,FN
iSH1474 (S12),56,142,0,5
iJN1462 (KT2440),50,129,13,11


iSH1474 accuracy: 97.53694581280789
iJN1462 accuracy: 88.17733990147784
